# Videoaula 10 Adequação de modelos e regressão não linear

## Exemplo 2: GT Auto

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from statsmodels.graphics.gofplots import ProbPlot
plt.style.use('seaborn')
plt.rc('axes', titlesize=10)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#Observar os dados
import io
#import pandas as pd
df = pd.read_csv(io.StringIO(uploaded['carst.csv'].decode('utf-8')))
df.head(30)


In [ ]:
sns.lmplot(x='cap_vol', y='consumo',hue='Etype', data=df);plt.grid(True)

In [ ]:
df1 = df[df.Etype != 1]
df1.head(15)

In [ ]:
df1 = pd.DataFrame(data=df1)
#definir a variável resposta
y = df1['consumo']
#definir a vriável preditora
x = df1['cap_vol']

In [ ]:
#@title Gráficos de diagnóstico 1 (y vs. x)
sns.lmplot(x='cap_vol', y='consumo', data=df1);plt.grid(True)

In [ ]:
#adicionar uma constante preditora
x = sm.add_constant(x)
#fit linear regression model
modelo = sm.OLS(y, x).fit() #atenção esta variável será usada no IC
print(modelo.summary())

In [ ]:
# parâmetros da reta
b=modelo.params
b0=b[0] #intercepto
b1=b[1] #inclinação
print('y= {0}{1} x'.format(b0,b1))

In [ ]:
x = df1['cap_vol']

In [ ]:
y_pred=list(modelo.predict())
#sns.resid(modelo)
resi=list(modelo.resid)

In [ ]:
#crear instancia influence
influence = modelo.get_influence()
#obter residuos standardizados
stdresid = list(influence.resid_studentized_internal)

In [ ]:
prop=np.divide(resi,stdresid) #e/e* para verificar a proporcionalidade entre o erro e o erro padronizado

In [ ]:
#@title Tabela geral
dft = pd.DataFrame(list(zip(x, y, y_pred, resi, stdresid, prop)),
               columns =["x","y","yc","e","e*","e/e*"])
dft 

In [ ]:
#@title Gráficos de diagnóstico 2 (y vs. yc)
sns.lmplot(x='y', y='yc', data=dft);plt.grid(True)

In [ ]:
#@title Gráficos de diagnóstico 3 (resíduos padronizados vs. x)
sns.scatterplot(x='x', y='e*', data=dft);plt.grid(True)
plt.xlabel('x= Capacidade Volumétrica')
plt.ylabel('e*= Resíduos padronizados')
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.show()

In [ ]:
#@title Gráficos de diagnóstico 4 (resíduos padronizados vs. y)
sns.scatterplot(x='y', y='e*', data=dft);plt.grid(True)
plt.xlabel('y= Consumo')
plt.ylabel('e*= Resíduos padronizados')
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.show()

In [ ]:
#@title Gráficos de diagnóstico 5 Probabilidade Normal
QQ = ProbPlot(influence.resid_studentized_internal)
plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
plot_lm_2.axes[0].set_xlabel('Percentil')
plot_lm_2.axes[0].set_ylabel('Resíduos padronizados')

In [ ]:
test = modelo.outlier_test()
print("test.columns:", test.columns)
print(test)
outliers = ((x[i],y[i]) for i,t in enumerate(test.iloc[:,2]) if t < 0.5)
print ('Outliers: ', list(outliers))

In [ ]:
plt.rcParams.update({'figure.figsize':(10,8), 'figure.dpi':100})
sns.lmplot(x='cap_vol', y='consumo',hue='brand/model/year', data=df1);

In [ ]:
a= modelo.conf_int(alpha=0.05, cols=None) #intervalo de confiança de b0 e b1
a

In [ ]:
display(a.iloc[0,:]) #intervalo de confiança b0
display(a.iloc[1,:]) #intervalo de confiança b0

## Exemplo 3
Nenhum aficionado de fritas gosta que fiquem encharcadas, por isso é importante encontrar características do processo de produção que as produzam com uma textura atraente. 
Os seguintes dados em que x=tempo de fritura (s) e y= teor de umidade (%) apareceram no artigo “Thermal and Physical Properties of Tortilla Chips as a Function of Frying Time” (J. of Food Processing and Preservation, 1995: 175–189).


a.   Construa um gráfico de dispersão de y versus x e comente.

b.   Construa um gráfico de dispersão dos pares (ln(x), ln(y)) e comente.

c.   Qual é a relação probabilística entre x e y sugerido pelo padrão linear no gráfico da parte (b)?

d.   Preveja o valor do teor de umidade ao fritar as batatas por 20s, de uma forma que transmite informações sobre confiabilidade e precisão.

e.   Analise os resíduos do ajuste do modelo de regressão linear para os dados transformados e comente.


In [ ]:
lstx = (5, 10, 15, 20, 25, 30, 45, 60)
lsty= (16.3, 9.7, 8.1, 4.2, 3.4, 2.9, 1.9, 1.3)
# Construir o DataFrame e nomear as colunas
dfries = pd.DataFrame(list(zip(lstx, lsty)),
               columns =["x","y"])
x=dfries['x']
y=dfries['y']
dfries.head()

In [ ]:
#@title Resposta a)
# Gráfico de dispersão
#plt.rcParams.update({'figure.figsize':(8,6), 'figure.dpi':100})
plt.scatter(dfries['x'],dfries['y'])
plt.xlabel('Tempo de fritura (s)');
plt.ylabel('Teor de umidade (%)')
plt.grid(True)
plt.show()

In [ ]:
#adicionar uma constante preditora
x = sm.add_constant(x)
#fit linear regression model
modelo = sm.OLS(y, x).fit() #atenção esta variável será usada no IC
print(modelo.summary())

In [ ]:
#@title Resposta b)
x=dfries['x']
y=dfries['y']
vtx=np.log(x)
vty=np.log(y)
plt.scatter(vtx,vty)
plt.xlabel('Tempo de fritura (s)');
plt.ylabel('Teor de umidade (%)')
plt.title('Gráfico com variáveis x e y transformadas')
plt.grid(True)
plt.show()

In [ ]:
dfries2 = pd.DataFrame(list(zip(x, y, vtx, vty)),
               columns =["x","y","ln_x","ln_y"])
dfries2.head()

In [ ]:
#@title Resposta c)
#adicionar uma constante preditora
vtx = sm.add_constant(vtx)
#fit linear regression model
modelo = sm.OLS(vty, vtx).fit() #atenção esta variável será usada no IC
print(modelo.summary())

In [ ]:
# parâmetros da reta
b=modelo.params
b0=b[0] #intercepto
b1=b[1] #inclinação
print('y= {0}{1} x'.format(b0,b1))

In [ ]:
from numpy.core.fromnumeric import mean
#@title Resposta d)


In [ ]:
#usar a tabela tstudent pata t
from scipy.stats import t
alpha = 0.05  # nível de significia = 5% 
df = len(x) - 2 # gl (n - 2)                                     
v = t.ppf(1 - alpha/2, df) 
print(f't=: {v}') 

Uma fórmula alternativa de SQE é

$SQE=∑(y_i - \hat{y}_i)$

$s^2=\frac{SQE}{n-2}$

$s=\sqrt(s^2)$

e $s_{\hat{Y}}=s\sqrt(\frac{1}{n}+\frac{(x^* -\bar{x})^2}{S_{XX}})$


In [ ]:
x=dfries['x']
y=dfries['y']
vtx=np.log(x)
vty=np.log(y)
y_pred=list(modelo.predict())
#sns.resid(modelo)
resi=(modelo.resid)
#SQE = residuos ao quadrado
sqe=np.sum(resi*resi) 
gl=modelo.df_resid  #gl=n-2
s2=sqe/gl
s=pow(s2,1/2)
sqe,s2,s

In [ ]:
vtxs=np.log(20) #x*=ln(20)
vtxm=vtx.mean()
n=modelo.df_resid + 2 #gl+2=n
xx=np.sum(vtx*vtx)
xau=np.sum(vtx)
sxx=xx-xau*xau/n
sy=s*pow(((1/n)+(vtxs-vtxm)*(vtxs-vtxm)/sxx),1/2)
sy


In [ ]:
#valor esperado de y para x=ln(20)
yln20=b0+b1*vtxs
yln20

In [ ]:
sya=pow((s2+sy*sy),1/2)
#IP do valor esperado para um valor específico de x 
IPyln20=([yln20-v*sya,yln20+v*sya])
import math
IPye200=pow(math.e,IPyln20[0])
IPye201=pow(math.e,IPyln20[1])
IPyln20,IPye200,IPye201

In [ ]:
#@title resposta e)
#x=dfries['x']
#y=dfries['y']
#vtx=np.log(x)
#vty=np.log(y)
#y_pred=list(modelo.predict())
##sns.resid(modelo)
resi=list(modelo.resid)
#crear instancia influence
influence = modelo.get_influence()
#obter residuos standardizados
stdresid = list(influence.resid_studentized_internal)
prop=np.divide(resi,stdresid) #e/e* para verificar a proporcionalidade entre o erro e o erro padronizado
dft2 = pd.DataFrame(list(zip(vtx, vty, y_pred, resi, stdresid, prop)),
               columns =["x","y","yc","e","e*","e/e*"])
dft2 

In [ ]:
#Gráficos de diagnóstico 5 Probabilidade Normal
QQ = ProbPlot(influence.resid_studentized_internal)
plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
plot_lm_2.axes[0].set_xlabel('Percentil')
plot_lm_2.axes[0].set_ylabel('Resíduos padronizados')